In [1]:
import numpy as np
import netCDF4 as nc
import gsw
from datetime import datetime,date
import os as os
from matplotlib.gridspec import GridSpec
from numpy.fft import fft,fft2,fftfreq
# importig movie py libraries
# from moviepy.editor import VideoClip
from scipy.interpolate import interp2d
# from moviepy.video.io.bindings import mplfig_to_npimage
from sklearn.linear_model import LinearRegression
from scipy import fftpack
from tqdm import tqdm
# analog data assimilation
from scipy.stats import linregress,norm
import dask as da
import xarray as xar
# from pydmd import DMD
import glob as glob




# The downloaded OCCIPUT files are modified according to our desired format, 

In [4]:
File=xar.open_mfdataset(glob.glob('Pacific_OCCIPUT*s.nc'))
Filemod=File.rename({'deptht':'depth','time_counter':'time','votemper':'TEMP','vosaline':'PSAL',"sossheig":"SSH"})
Filemod=Filemod.assign({"latitude":Filemod['latitude'].isel(time=0).mean('x'),"longitude":Filemod['longitude'].where(Filemod['longitude']>0,Filemod['longitude']+360).isel(time=0).mean('y'),})
Filemod=Filemod.drop(['time_centered_bounds','time_counter_bounds','time_centered']).swap_dims({"x":"longitude",'y':'latitude'})
Filemod=Filemod.assign({'longitude':Filemod['longitude'].where(np.where(Filemod['longitude'])[0]!=60., -179.81245+360)})

Filemod.to_netcdf('Pacific_OCCIPUT_Full.nc')

In [5]:

Filemod=xar.open_mfdataset('Pacific_OCCIPUT_Full.nc')
Filemod=Filemod.where(Filemod['longitude']<=290,drop=True)
Filemod=Filemod.where((Filemod.isnull().rolling({'time':4},center=True,min_periods=2).sum('time').isin([0,1,2,3])).all('time')).interpolate_na('time',limit=3)



# The gridded temperature and salinity fields are coarsed down to 3° cells. A climatology is evaluated over the 1995-2015 period. The fields are saved.

In [7]:
RC = Filemod.coarsen({'longitude':3,'latitude':3},boundary="trim",side='left').mean().transpose("time","depth","latitude","longitude")

JULTIME = (RC['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64')

RC = RC.assign({'time2':(JULTIME)})

RC.coords['time2']=RC.coords['time']
RC.coords['time']=JULTIME



RC = RC.merge(RC[['TEMP','PSAL']].where(RC['time2'].dt.year>=1995,drop=True).groupby("time2.month").mean(skipna=False).rename({'TEMP':'T_Climato','PSAL':'S_Climato'}))


res2 = 9

RC = RC.merge((RC[['TEMP','PSAL']].where(RC['time2'].dt.year>=1995,drop=True).groupby('time2.month')-RC[['TEMP','PSAL']].where(RC['time2'].dt.year>=1995,drop=True).groupby('time2.month').mean(skipna=False)).polyfit(deg=1,dim='time'))
              

RC.to_netcdf('Pacific_OCCIPUT_Full2.nc','w',format='NETCDF4')

In [2161]:
RC=xar.open_mfdataset('Pacific_OCCIPUT_Full2.nc')
RC=RC.where(RC['time2'].dt.year>=1995,drop=True).isel(depth=[0,3,4,5,6,7,8])

RC['TEMP_polyfit_coefficients'] = RC['TEMP_polyfit_coefficients'].isel(time=0)
RC['T_Climato'] = RC['T_Climato'].isel(time=0)
RC['S_Climato'] = RC['S_Climato'].isel(time=0)
RC['PSAL_polyfit_coefficients'] = RC['PSAL_polyfit_coefficients'].isel(time=0)

# RC = RC.assign({'TEMP_polyfit_coefficients':RC['TEMP_polyfit_coefficients'].where(RC['TEMP'].isel(time=0).notnull()),'PSAL_polyfit_coefficients':RC['PSAL_polyfit_coefficients'].where(RC['PSAL'].isel(time=0).notnull())})


# Temperature:  The files of the synthetic observations are opened. They are then standardized over the desired depth levels. 
# To evaluate the datasets for all the decades, timemin and timemax have to be modified manually.

In [81]:
timemin=1990
timemax=timemin+9
ENtemp = xar.open_mfdataset('ENtemp'+str(timemin)[-2:]+'Pacific_OCCIPUT.nc',parallel=True)

switch = False 
for zepth in [5,10,50,100,300,500,700,1000,1500,2000]:
    
    if zepth == 2000:
        bnd1,bnd2 = 1750,2250
        
    if zepth == 1500:
        bnd1,bnd2 = 1250,1750
        
    if zepth == 1000:
        bnd1,bnd2 = 850,1250
    elif (zepth==700):
        bnd1,bnd2 = 600,800
    elif (zepth==500):
        bnd1,bnd2 = 450,600
    elif (zepth==400):
        bnd1,bnd2 = 350,450
    elif (zepth==300):
        bnd1,bnd2 = 250,350
    elif (zepth==200):
        bnd1,bnd2 = 150,250
    elif (zepth==100):
        bnd1,bnd2 = 75,150
    elif (zepth==50):
        bnd1,bnd2 = 25,75
    elif (zepth==10):
        bnd1,bnd2 = 7,25
    else:
        bnd1,bnd2 = 0,7
        
    print(zepth)
    
    
    argumt=da.array.argmin(da.array.where(zepth-ENtemp['DEPTH'].fillna(1e5)>=0,zepth-ENtemp['DEPTH'].fillna(1e5),1e5),1).compute()
    born1=da.array.all(zepth-ENtemp['DEPTH'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')<0,axis=1)
    born2=da.array.all(zepth-ENtemp['DEPTH'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')>0,axis=1)

    Nprs = ENtemp.coords['N_PROF'].size
    Nzz = ENtemp.coords['N_LEVELS'].size

    z0,z1=ENtemp['DEPTH'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENtemp['DEPTH'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>149,149,(1+argumt))]
    quality = (ENtemp['POTM_QC']==1.)&(ENtemp['POTM_LEVEL_QC']<=2.)
    T0,T1=ENtemp['POTM'].where(quality).data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENtemp['POTM'].where(quality).data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>149,149,(1+argumt))]

    born3 = np.array([born1, born2, (z0 <bnd1), (z1>bnd2) ,(z0 > z1) ,(zepth<z0) ,(zepth>z1)]).any(axis=0)



    #     born3 = (z0-z1>=zscl)
    T0=np.where(born3,np.nan,T0)
    T1=np.where(born3,np.nan,T1)
    if switch : 
        ENewtemp = np.concatenate((ENewtemp,(np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))])),axis=0)
        # QC_z = np.concatenate((QC_z,np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]),axis=0)
    if not switch: 
        ENewtemp = np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))]
        # QC_z = np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]
        switch = True

       #
#      


5
10
50
100
300
500
700
1000
1500
2000


# The super-profiles are created for obserations with daily frequency measurements (moorings, mostly). The observation files are saved. 

In [82]:

ENtemp2 = xar.Dataset( 
        {
            'POTM': (['depth','N_PROF'],ENewtemp),
            # 'QC' : (['depth','N_PROF'],QC_z.compute())
        },
        coords = {
            'JULD': (['N_PROF'],ENtemp['JULD'].values),
            'LATITUDE': (['N_PROF'],ENtemp['LATITUDE'].values),
            'LONGITUDE': (['N_PROF'],ENtemp['LONGITUDE'].values),
            'Depth': (['depth'],RC['depth'].values),
            'STATION_IDENTIFIER' : (['N_PROF'],ENtemp['STATION_IDENTIFIER'].values),
            'STATION_NUMBER' : (['N_PROF'], ENtemp['STATION_TYPE'].values)
        }
)





indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)

indexes = np.char.array(['820'] ).zfill(3)

condit = ((ENtemp2['STATION_NUMBER'].dropna('N_PROF').str.strip().astype('str').isin(indexes)))


moorObs = ENtemp2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['STATION_IDENTIFIER'])

timebins=np.arange(np.datetime64(str(timemin)+'-01-01'),np.datetime64(str(timemax)+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['STATION_IDENTIFIER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor[1:] : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['STATION_IDENTIFIER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1

    
    
    
    ENtemp3 = xar.concat((ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['STATION_NUMBER','STATION_IDENTIFIER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENtemp3 =ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['STATION_NUMBER','STATION_IDENTIFIER'])
    
    
    
    
    
ENtemp3.to_netcdf('ENtemp_NoInterp'+str(timemin)[-2:]+'NoFMoor_OCCIPUT.nc','w')
ENtemp2.to_netcdf('ENtemp_NoInterp'+str(timemin)[-2:]+'_OCCIPUT.nc','w')

# Salinity:  The files of the synthetic observations are opened. They are then standardized over the desired depth levels. 
# To evaluate the datasets for all the decades, timemin and timemax have to be modified manually.

In [89]:
timemin=1990
timemax=timemin+9
ENsal = xar.open_mfdataset('ENsal'+str(timemin)[-2:]+'Pacific_OCCIPUT.nc',parallel=True)

switch = False 
for zepth in [5,10,50,100,300,500,700,1000,1500,2000]:
    
    if zepth == 2000:
        bnd1,bnd2 = 1750,2250
        
    if zepth == 1500:
        bnd1,bnd2 = 1250,1750
        
    if zepth == 1000:
        bnd1,bnd2 = 850,1250
    elif (zepth==700):
        bnd1,bnd2 = 600,800
    elif (zepth==500):
        bnd1,bnd2 = 450,600
    elif (zepth==400):
        bnd1,bnd2 = 350,450
    elif (zepth==300):
        bnd1,bnd2 = 250,350
    elif (zepth==200):
        bnd1,bnd2 = 150,250
    elif (zepth==100):
        bnd1,bnd2 = 75,150
    elif (zepth==50):
        bnd1,bnd2 = 25,75
    elif (zepth==10):
        bnd1,bnd2 = 7,25
    else:
        bnd1,bnd2 = 0,7
        
    print(zepth)
    
    argumt=da.array.argmin(da.array.where(zepth-ENsal['DEPTH'].fillna(1e5)>=0,zepth-ENsal['DEPTH'].fillna(1e5),1e5),1).compute()
    born1=da.array.all(zepth-ENsal['DEPTH'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')<0,axis=1)
    born2=da.array.all(zepth-ENsal['DEPTH'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')>0,axis=1)

    Nprs = ENsal.coords['N_PROF'].size
    Nzz = ENsal.coords['N_LEVELS'].size
    
    z0,z1=ENsal['DEPTH'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENsal['DEPTH'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>149,149,(1+argumt))]
    quality = (ENsal['PSAL_QC']==1.)&(ENsal['PSAL_LEVEL_QC']<=2.)
    T0,T1=ENsal['PSAL'].where(quality).data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENsal['PSAL'].where(quality).data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>149,149,(1+argumt))]

    born3 = np.array([born1, born2, (z0 <bnd1), (z1>bnd2) ,(z0 > z1) ,(zepth<z0) ,(zepth>z1)]).any(axis=0)

    #     born3 = (z0-z1>=zscl)
    T0=np.where(born3,np.nan,T0)
    T1=np.where(born3,np.nan,T1)
    if switch : 
        ENewtemp = np.concatenate((ENewtemp,(np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))])),axis=0)
        # QC_z = np.concatenate((QC_z,np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]),axis=0)
    if not switch: 
        ENewtemp = np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))]
        # QC_z = np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]
        switch = True

       #
#      


5
10
50
100
300
500
700
1000
1500
2000


# The super-profiles are created for obserations with daily frequency measurements (moorings, mostly). The observation files are saved. 

In [226]:

ENsal2 = xar.Dataset( 
        {
            'PSAL': (['depth','N_PROF'],ENewtemp),
            # 'QC' : (['depth','N_PROF'],QC_z.compute())
        },
        coords = {
            'JULD': (['N_PROF'],ENsal['JULD'].values),
            'LATITUDE': (['N_PROF'],ENsal['LATITUDE'].values),
            'LONGITUDE': (['N_PROF'],ENsal['LONGITUDE'].values),
            'Depth': (['depth'],RC['depth'].values),
            'STATION_IDENTIFIER' : (['N_PROF'],ENsal['STATION_IDENTIFIER'].values),
            'STATION_NUMBER' : (['N_PROF'], ENsal['STATION_TYPE'].values)
        }
)





indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)

indexes = np.char.array(['820'] ).zfill(3)

condit = ((ENsal2['STATION_NUMBER'].dropna('N_PROF').str.strip().astype('str').isin(indexes)))


moorObs = ENsal2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['STATION_IDENTIFIER'])

timebins=np.arange(np.datetime64(str(timemin)+'-01-01'),np.datetime64(str(timemax)+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['STATION_IDENTIFIER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor[1:] : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['STATION_IDENTIFIER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1

    
    
    ENsal3 = xar.concat((ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['STATION_NUMBER','STATION_IDENTIFIER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENsal3 =ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['STATION_NUMBER','STATION_IDENTIFIER'])
    
    
    
    
    
ENsal3.to_netcdf('ENsal_NoInterp'+str(timemin)[-2:]+'NoFMoor_OCCIPUT.nc','w')
ENsal2.to_netcdf('ENsal_NoInterp'+str(timemin)[-2:]+'_OCCIPUT.nc','w')

ValueError: conflicting sizes for dimension 'depth': length 6 on 'Depth' and length 10 on {'depth': 'PSAL', 'N_PROF': 'PSAL'}